In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lending-club-dataset/loan.csv
/kaggle/input/lending-club-dataset/Data_Dictionary.xlsx


In [2]:
loan = pd.read_csv("/kaggle/input/lending-club-dataset/loan.csv")
loan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Columns: 111 entries, id to total_il_high_credit_limit
dtypes: float64(74), int64(13), object(24)
memory usage: 33.6+ MB


/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (47) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
print(f"Shape before dropping columns with nulls > 10% \t:\t{loan.shape}")
null_50_cols = loan.columns[loan.isna().sum()/len(loan.index) *100 > 10]
print(f"Number of Columns with nulls > 50% \t:\t{len(null_50_cols)}")
#Dropping Columns with nulls > 50%
loan.drop(null_50_cols, axis =1, inplace=True)
print(f"Shape after dropping columns with nulls > 50% \t:\t{loan.shape}")

Shape before dropping columns with nulls > 10% 	:	(39717, 111)
Number of Columns with nulls > 50% 	:	58
Shape after dropping columns with nulls > 50% 	:	(39717, 53)


In [4]:
#Columns with only one value in them
single_value_columns = []
for i in loan.columns:
    if loan[i].dropna().nunique() == 1:
        single_value_columns.append(i)
print(f'Columns with only one value \t:\t {single_value_columns}')

Columns with only one value 	:	 ['pymnt_plan', 'initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens']


In [5]:
print(f"Shape before dropping columns with only one value \t:\t{loan.shape}")
print(f"Number of Columns with only one value \t:\t{len(single_value_columns)}")
loan.drop(single_value_columns, axis =1, inplace=True)
print(f"Shape after dropping columns only one value \t:\t{loan.shape}")

Shape before dropping columns with only one value 	:	(39717, 53)
Number of Columns with only one value 	:	9
Shape after dropping columns only one value 	:	(39717, 44)


In [6]:
#Number of Duplicated Rows
loan.duplicated().sum()

0

In [7]:
loan.iloc[:,:10]

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5
...,...,...,...,...,...,...,...,...,...,...
39712,92187,92174,2500,2500,1075.0,36 months,8.07%,78.42,A,A4
39713,90665,90607,8500,8500,875.0,36 months,10.28%,275.38,C,C1
39714,90395,90390,5000,5000,1325.0,36 months,8.07%,156.84,A,A4
39715,90376,89243,5000,5000,650.0,36 months,7.43%,155.38,A,A2


In [8]:
#Of these 10 columns, id and member_id are not relevant for prediction. Term and int_rate need to be
#Formatted and changed into int/float. grade should be dropped since sub grade makes it redundant. sub_grade
#Should be encoded

In [9]:
print(f"Shape before dropping grade \t:\t{loan.shape}")
loan.drop(['grade'], axis = 1, inplace = True)
print(f"Shape after dropping grade \t:\t{loan.shape}")

Shape before dropping grade 	:	(39717, 44)
Shape after dropping grade 	:	(39717, 43)


In [10]:
#Formatting int_rate
loan['int_rate'] = loan['int_rate'].str.replace("%","").astype(float)

In [11]:
loan['term'].unique()

array([' 36 months', ' 60 months'], dtype=object)